In [1]:
!pip install python-binance

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import binance_framework as bf
from collections import deque
import random

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [3]:
#Note: BCH HAS MANY NULL VALUES, BETTER NOT INCLUDE IT
currencies = ['ETHUSDT']
coins = bf.get_data_by_intervals(50,currencies,bf.INTERVAL_1MINUTE)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24951 entries, 0 to 24950
Data columns (total 12 columns):
ETHUSDT Open Time                       24951 non-null object
ETHUSDT Open                            24951 non-null object
ETHUSDT High                            24951 non-null object
ETHUSDT Low                             24951 non-null object
ETHUSDT Close                           24951 non-null object
ETHUSDT Volume                          24951 non-null object
ETHUSDT Close Time                      24951 non-null object
ETHUSDT Quote asset volume              24951 non-null object
ETHUSDT n_trades                        24951 non-null object
ETHUSDT Taker buy base asset volume     24951 non-null object
ETHUSDT Taker buy quote asset volume    24951 non-null object
ETHUSDT Ignore                          24951 non-null object
dtypes: object(12)
memory usage: 2.3+ MB


In [0]:
coins.info()

In [4]:
bf.show_time_skips(coins, currencies)

ETHUSDT  Time difference:  21300000
between indexes:  4715 , 4716
gap starts at:  1582112100000
gap ends at:    1582133400000


ETHUSDT  Time difference:  7740000
between indexes:  24387 , 24388
gap starts at:  1583313660000
gap ends at:    1583321400000


